---
jupyter: python3
toc: true
toc-depth: 3
number-sections: true
title: Pandas_02_Indexing, 값 변경, 추가
date: 2021-11-05 00:01
categories: pandas
author: limyj0708
comments:
  giscus:
    repo: quarto-dev/quarto-docs
format:
    html:
        page-layout: full
---

In [72]:
#| code-fold: true
#| code-summary: 'import codes'
import pandas as pd
import numpy as np
import copy
from IPython.display import display, HTML, display_html

df = pd.read_parquet('df.parquet', engine='pyarrow') 

def display_multiple_dfs(dfs:list, styles, margin=10):
    display_target = f"""<div>"""
    for each_df in dfs:
        each_df_html = each_df[0].style.set_table_attributes(f"style='display:inline;margin:{margin}px'").to_html()
        in_div = f"""
        <div style="float: left; padding-right: 20px">
            <p><b>{each_df[1]}</b></p>
            {each_df_html}
        </div>
        """
        display_target += in_div
    #print(display_target)
    display_html(display_target, raw = True)

# loc : 라벨 인덱싱

In [76]:
print(type(df.loc[0]))
df.loc[0]
# loc의 첫 번째 인자는 '행 라벨' 이다.
# 그래서 0을 넣으면, index가 0인 행을 series로 반환하고 있다.

<class 'pandas.core.series.Series'>


A     1.0
B     4.0
BB    NaN
C     0.0
D     NaN
Name: 0, dtype: float64

In [77]:
print(type(df.loc[0, 'A']))
df.loc[0, 'A']
# 두 번째 인자는 컬럼명이다.

<class 'numpy.int64'>


1

In [78]:
df.loc[[0,1,2,3], ['A','B']]
# 이런 식으로 접근하면, 다중 컬럼과 행을 데이터프레임으로 가져올 수 있다.

,A,B
0,1,4
1,3,4
2,1,6
3,1,2


In [79]:
df.loc[df.index[0:3], ['A','B']]
# df.index로도 접근 가능

,A,B
0,1,4
1,3,4
2,1,6


In [80]:
df.loc[df['B'] == 4]
# row에 값 조건을 걸 수도 있다.

,A,B,BB,C,D
0,1,4,NaN,0.0,NaN
1,3,4,NaN,0.0,NaN


In [81]:
df.loc[df['B'] == 4, df.columns.str.contains('B')]
# 컬럼 이름에도 조건을 걸 수 있다. 위의 경우, 컬럼 이름에 B를 포함하는 컬럼만 가져옴.

,B,BB
0,4,NaN
1,4,NaN


In [82]:
df.loc[:,df.columns.str.contains('B')]
# 행 조건 자리에 :를 넣으면, 행에 대해서는 전체를 다 가져오라는 뜻이다.

,B,BB
0,4,NaN
1,4,NaN
2,6,NaN
3,2,3.0


In [83]:
print(df.columns) # 컬럼명을 가져옴
print(df.columns.str)
print(df.columns.str.contains('B')) # boolean indexing이 가능한 형태가 된다.
print(type(df.columns.str.contains('B'))) # 결과물은 false와 true가 들어간 ndarray
print(df.columns.str.startswith('A')) # 이렇게 하면 A로 시작하는 컬럼을 가져올 수 있음
# 결론은, 다른 외부 함수를 사용해서 어쩄든 boolean 타입 값이 담긴 리스트를 만들면, loc에 넣어서 boolean indexing이 가능하다는 것.

Index(['A', 'B', 'BB', 'C', 'D'], dtype='object')
[False  True  True False False]
<class 'numpy.ndarray'>
[ True False False False False]


In [84]:
df.loc[:,'new'] = 3
df
# loc으로도 기존에 없던 새 컬럼을 추가할 수 있음

,A,B,BB,C,D,new
0,1,4,NaN,0.0,NaN,3
1,3,4,NaN,0.0,NaN,3
2,1,6,NaN,NaN,NaN,3
3,1,2,3.0,4.0,5.0,3


In [85]:
df.loc[4] = [1] * len(df.columns)
df.loc[99] = [1] * len(df.columns)
df.loc['cool'] = [22] * len(df.columns)
# dataframe에 행을 추가함. index가 늘어난다.
df

,A,B,BB,C,D,new
0,1,4,NaN,0.0,NaN,3
1,3,4,NaN,0.0,NaN,3
2,1,6,NaN,NaN,NaN,3
3,1,2,3.0,4.0,5.0,3
4,1,1,1.0,1.0,1.0,1
99,1,1,1.0,1.0,1.0,1
cool,22,22,22.0,22.0,22.0,22


# iloc : 위치 인덱싱

In [86]:
df.iloc[0:2,0:4]
# 기본적 동작은 loc과 동일하나, 받는 인자가 라벨이 아니고 '위치'다.

,A,B,BB,C
0,1,4,NaN,0.0
1,3,4,NaN,0.0


In [87]:
df.iloc[4:7,0:4]
# 위치를 받기 때문에, index는 99여도 5번째 줄로 인식됨

,A,B,BB,C
4,1,1,1.0,1.0
99,1,1,1.0,1.0
cool,22,22,22.0,22.0


In [88]:
df.iloc[7] = [2] * len(df.columns)
# IndexError: iloc cannot enlarge its target object
# 위치를 인자로 받기 때문에, 새로운 컬럼, 행을 만든다거나 하는 행위는 불가능하다.

IndexError: iloc cannot enlarge its target object

# at : 스칼라값 접근

In [89]:
df.at[1,'A']
# 한 번에 1개의 스칼라값에만 접근 가능
# 여러 개의 값에 접근하려고 범위를 지정하면, 에러를 출력한다.
# 단일 값에 접근하는 목적이라면 loc보다 훨씬 빠름

3

In [90]:
df.at[1,'A'] = 100
df
# 값을 딱 하나만 바꾸고 싶다! 라고 하면 at을 활용해보자.

,A,B,BB,C,D,new
0,1,4,NaN,0.0,NaN,3
1,100,4,NaN,0.0,NaN,3
2,1,6,NaN,NaN,NaN,3
3,1,2,3.0,4.0,5.0,3
4,1,1,1.0,1.0,1.0,1
99,1,1,1.0,1.0,1.0,1
cool,22,22,22.0,22.0,22.0,22


In [91]:
df.at[99, 'new']
# 그 이외에는 label base인 것이 loc과 똑같음

1

# iat : iloc의 스칼라 버전

In [92]:
df.iat[4,2]
# iloc의 스칼라 버전.
# 이외의 동작은 at과 같다.

1.0

In [93]:
df.iat[df.index.get_loc('cool'),df.columns.get_loc('new')]
# get_loc을 쓰면, 해당 인덱스와 컬럼의 위치를 반환받을 수 있음.
# 그럼 인덱스와 컬럼의 이름으로도 iat, iloc을 이용 가능

22

# map : Series의 원소 하나하나에 함수 적용
 
- map함수는 DataFrame 타입이 아니라, 반드시 Series 타입에서만 사용해야 한다.
- Series를 한마디로 정의하면 딱 이거다.
    - 값(value) + 인덱스(index) = 시리즈 클래스(Series)
- Series는 NumPy에서 제공하는 1차원 배열과 비슷하지만 각 데이터의 의미를 표시하는 인덱스(index)를 붙일 수 있다. 하지만 데이터 자체는 그냥 값(value)의 1차원 배열이다.
- map함수는 Series의 이러한 값 하나하나에 접근하면서 해당 함수를 수행한다.

In [94]:
import math as m # sqrt 함수 사용을 위해 부름
# http://www.leejungmin.org/post/2018/04/21/pandas_apply_and_map/
df["map_b"] = df["B"].map(lambda x : m.sqrt(x)) 
# B컬럼의 값 하나하나에 sqrt 함수를 적용한 결과를 map_b 컬럼으로 추가
df

,A,B,BB,C,D,new,map_b
0,1,4,NaN,0.0,NaN,3,2.000000
1,100,4,NaN,0.0,NaN,3,2.000000
2,1,6,NaN,NaN,NaN,3,2.449490
3,1,2,3.0,4.0,5.0,3,1.414214
4,1,1,1.0,1.0,1.0,1,1.000000
99,1,1,1.0,1.0,1.0,1,1.000000
cool,22,22,22.0,22.0,22.0,22,4.690416


# apply : 커스텀 함수에 복수 개의 컬럼이 필요하다면
- 커스텀 함수를 사용하기 위해 DataFrame에서 복수 개의 컬럼이 필요하다면, apply 함수를 사용해야 한다.

In [95]:
import math as m # sqrt 함수 사용을 위해 부름
# 두 컬럼의 제곱근의 값을 각각 곱하는 함수
def sqrt_multi(x,y):
    return m.sqrt(x) * m.sqrt(y)

In [96]:
df.loc[:,'new'] = df.apply(lambda x : sqrt_multi(x['A'], x['B']), axis=1) # axis=1 이면 각 열의 원소에 대해 연산 수행
df

,A,B,BB,C,D,new,map_b
0,1,4,NaN,0.0,NaN,2.000000,2.000000
1,100,4,NaN,0.0,NaN,20.000000,2.000000
2,1,6,NaN,NaN,NaN,2.449490,2.449490
3,1,2,3.0,4.0,5.0,1.414214,1.414214
4,1,1,1.0,1.0,1.0,1.000000,1.000000
99,1,1,1.0,1.0,1.0,1.000000,1.000000
cool,22,22,22.0,22.0,22.0,22.000000,4.690416


In [97]:
df["apply_bb_d"] = df.apply(lambda x : sqrt_multi(x['BB'], x['B']), axis=1) # axis=1 이면 각 열의 원소에 대해 연산 수행
df # NaN과의 연산은 NaN이 됨을 참고하자.

,A,B,BB,C,D,new,map_b,apply_bb_d
0,1,4,NaN,0.0,NaN,2.000000,2.000000,NaN
1,100,4,NaN,0.0,NaN,20.000000,2.000000,NaN
2,1,6,NaN,NaN,NaN,2.449490,2.449490,NaN
3,1,2,3.0,4.0,5.0,1.414214,1.414214,2.44949
4,1,1,1.0,1.0,1.0,1.000000,1.000000,1.00000
99,1,1,1.0,1.0,1.0,1.000000,1.000000,1.00000
cool,22,22,22.0,22.0,22.0,22.000000,4.690416,22.00000


# index, columns로 index와 컬럼명 직접 지정

In [98]:
print(df.columns)
print(type(df.columns))
print(df.index)
print(type(df.index))
print(df.columns[2]) # 위치값으로 개별 요소에 접근 가능
print(df.index[6])
df

Index(['A', 'B', 'BB', 'C', 'D', 'new', 'map_b', 'apply_bb_d'], dtype='object')
<class 'pandas.core.indexes.base.Index'>
Index([0, 1, 2, 3, 4, 99, 'cool'], dtype='object')
<class 'pandas.core.indexes.base.Index'>
BB
cool


,A,B,BB,C,D,new,map_b,apply_bb_d
0,1,4,NaN,0.0,NaN,2.000000,2.000000,NaN
1,100,4,NaN,0.0,NaN,20.000000,2.000000,NaN
2,1,6,NaN,NaN,NaN,2.449490,2.449490,NaN
3,1,2,3.0,4.0,5.0,1.414214,1.414214,2.44949
4,1,1,1.0,1.0,1.0,1.000000,1.000000,1.00000
99,1,1,1.0,1.0,1.0,1.000000,1.000000,1.00000
cool,22,22,22.0,22.0,22.0,22.000000,4.690416,22.00000


In [99]:
df.columns = ['가', '나', '다', '라', '마', '바', '사', '아'] 
# df.columns에 직접 컬럼명 리스트를 할당하여 컬럼명 변경 가능
# 기존 컬럼 수와 같은 길이의 리스트를 넣지 않으면 오류가 발생함
print(df.columns)
print(type(df.columns))

Index(['가', '나', '다', '라', '마', '바', '사', '아'], dtype='object')
<class 'pandas.core.indexes.base.Index'>


In [100]:
df.index = [1,2,3,4,5,6,7] 
# df.columns에 직접 컬럼명 리스트를 할당하여 컬럼명 변경 가능
print(df.index)
print(type(df.index))

Int64Index([1, 2, 3, 4, 5, 6, 7], dtype='int64')
<class 'pandas.core.indexes.numeric.Int64Index'>


In [101]:
df

,가,나,다,라,마,바,사,아
1,1,4,NaN,0.0,NaN,2.000000,2.000000,NaN
2,100,4,NaN,0.0,NaN,20.000000,2.000000,NaN
3,1,6,NaN,NaN,NaN,2.449490,2.449490,NaN
4,1,2,3.0,4.0,5.0,1.414214,1.414214,2.44949
5,1,1,1.0,1.0,1.0,1.000000,1.000000,1.00000
6,1,1,1.0,1.0,1.0,1.000000,1.000000,1.00000
7,22,22,22.0,22.0,22.0,22.000000,4.690416,22.00000


# set_index로 index 설정
```DataFrame.set_index(keys, drop=True, append=False, inplace=False)```

- keys에는 index로 할당하고자 하는 열의 레이블을 입력한다.
  - multi-index를 하고 싶으면, ['가', '나'] 이렇게 열 레이블 배열을 입력한다.
- drop : index로 할당한 열을 삭제할까요?
- append : 기존에 존재하던 index를 삭제할까요?
- inplace : 원본 데이터프레임을 변경할까요?

In [102]:
df

,가,나,다,라,마,바,사,아
1,1,4,NaN,0.0,NaN,2.000000,2.000000,NaN
2,100,4,NaN,0.0,NaN,20.000000,2.000000,NaN
3,1,6,NaN,NaN,NaN,2.449490,2.449490,NaN
4,1,2,3.0,4.0,5.0,1.414214,1.414214,2.44949
5,1,1,1.0,1.0,1.0,1.000000,1.000000,1.00000
6,1,1,1.0,1.0,1.0,1.000000,1.000000,1.00000
7,22,22,22.0,22.0,22.0,22.000000,4.690416,22.00000


In [103]:
df.set_index('가') # 기본값

,나,다,라,마,바,사,아
가,,,,,,,
1,4,NaN,0.0,NaN,2.000000,2.000000,NaN
100,4,NaN,0.0,NaN,20.000000,2.000000,NaN
1,6,NaN,NaN,NaN,2.449490,2.449490,NaN
1,2,3.0,4.0,5.0,1.414214,1.414214,2.44949
1,1,1.0,1.0,1.0,1.000000,1.000000,1.00000
1,1,1.0,1.0,1.0,1.000000,1.000000,1.00000
22,22,22.0,22.0,22.0,22.000000,4.690416,22.00000


In [104]:
df.set_index('가', drop=False) # index로 선택된 열 삭제 안 함

,가,나,다,라,마,바,사,아
가,,,,,,,,
1,1,4,NaN,0.0,NaN,2.000000,2.000000,NaN
100,100,4,NaN,0.0,NaN,20.000000,2.000000,NaN
1,1,6,NaN,NaN,NaN,2.449490,2.449490,NaN
1,1,2,3.0,4.0,5.0,1.414214,1.414214,2.44949
1,1,1,1.0,1.0,1.0,1.000000,1.000000,1.00000
1,1,1,1.0,1.0,1.0,1.000000,1.000000,1.00000
22,22,22,22.0,22.0,22.0,22.000000,4.690416,22.00000


In [105]:
df.set_index('가', append=True) # 기존 index 삭제 안 함

,,나,다,라,마,바,사,아
,가,,,,,,,
1,1,4,NaN,0.0,NaN,2.000000,2.000000,NaN
2,100,4,NaN,0.0,NaN,20.000000,2.000000,NaN
3,1,6,NaN,NaN,NaN,2.449490,2.449490,NaN
4,1,2,3.0,4.0,5.0,1.414214,1.414214,2.44949
5,1,1,1.0,1.0,1.0,1.000000,1.000000,1.00000
6,1,1,1.0,1.0,1.0,1.000000,1.000000,1.00000
7,22,22,22.0,22.0,22.0,22.000000,4.690416,22.00000


In [106]:
df.set_index(['가','나']) # 동시에 여러 열을 index로 설정하기

다     라     마          바         사         아
가   나                                                  
1   4    NaN   0.0   NaN   2.000000  2.000000       NaN
100 4    NaN   0.0   NaN  20.000000  2.000000       NaN
1   6    NaN   NaN   NaN   2.449490  2.449490       NaN
    2    3.0   4.0   5.0   1.414214  1.414214   2.44949
    1    1.0   1.0   1.0   1.000000  1.000000   1.00000
    1    1.0   1.0   1.0   1.000000  1.000000   1.00000
22  22  22.0  22.0  22.0  22.000000  4.690416  22.00000

# reset_index로 index 초기화
```DataFrame.reset_index(drop=False, inplace=False)```

- 기존에 있던 index 대신에, 0부터 시작하여 1씩 늘어나는 정수 index를 추가한다.
- drop : 기존에 index였던 열을 삭제할까요?
- inplace : 원본 데이터프레임을 변경할까요? 

In [107]:
df.reset_index()

,index,가,나,다,라,마,바,사,아
0,1,1,4,NaN,0.0,NaN,2.000000,2.000000,NaN
1,2,100,4,NaN,0.0,NaN,20.000000,2.000000,NaN
2,3,1,6,NaN,NaN,NaN,2.449490,2.449490,NaN
3,4,1,2,3.0,4.0,5.0,1.414214,1.414214,2.44949
4,5,1,1,1.0,1.0,1.0,1.000000,1.000000,1.00000
5,6,1,1,1.0,1.0,1.0,1.000000,1.000000,1.00000
6,7,22,22,22.0,22.0,22.0,22.000000,4.690416,22.00000


In [108]:
df.reset_index(drop=True)

,가,나,다,라,마,바,사,아
0,1,4,NaN,0.0,NaN,2.000000,2.000000,NaN
1,100,4,NaN,0.0,NaN,20.000000,2.000000,NaN
2,1,6,NaN,NaN,NaN,2.449490,2.449490,NaN
3,1,2,3.0,4.0,5.0,1.414214,1.414214,2.44949
4,1,1,1.0,1.0,1.0,1.000000,1.000000,1.00000
5,1,1,1.0,1.0,1.0,1.000000,1.000000,1.00000
6,22,22,22.0,22.0,22.0,22.000000,4.690416,22.00000


# between
```Series.between(left, right, inclusive='both')```
- left : scalar or list-like
  - Left boundary.
- right : scalar or list-like
  - Right boundary.
- inclusive{“both”, “neither”, “left”, “right”}
  - Include boundaries. Whether to set each bound as closed or open
  - 어느 쪽에 부등호를 지정할 것인가?

In [109]:
dict_sample = [
    {'base_date':np.datetime64('2023-01-03'), 'num':10}
  , {'base_date':np.datetime64('2023-01-03'), 'num':11}
  , {'base_date':np.datetime64('2023-01-04'), 'num':12}
  , {'base_date':np.datetime64('2023-01-06'), 'num':13}
  , {'base_date':np.datetime64('2023-01-07'), 'num':14}
  , {'base_date':np.datetime64('2023-01-08'), 'num':15}
]
df_sample = pd.DataFrame.from_dict(dict_sample)

In [110]:
date_mask = df_sample['base_date'].between(np.datetime64('2023-01-03'), np.datetime64('2023-01-05'), inclusive='left')
df_sample.loc[date_mask]

,base_date,num
0,2023-01-03,10
1,2023-01-03,11
2,2023-01-04,12


In [111]:
# groupby, agg와 함께 사용하면 이렇게 집계 가능
df_sample.loc[date_mask].groupby(by=['base_date']).agg({'num':np.sum})

,num
base_date,
2023-01-03,21
2023-01-04,12


# rename : 인덱스, 컬럼 이름 변경
```DataFrame.rename(mapper=None, *, index=None, columns=None, axis=None, copy=None, inplace=False, level=None, errors='ignore')```

- mapper : dict-like or funtion
  - 이름 A를 B로 바꾸는 매핑 정보가 담겨 있는 딕셔너리나, A를 넣으면 B가 반환되는 함수.
- index : dict-like or funtion
  - mapper, axis=0과 index=mapper는 같은 의미이다.
- columns : dict-like or funtion
  - mapper, axis=1과 index=columns는 같은 의미이다.
- axis : {0 or ‘index’, 1 or ‘columns’}, default 0
  - mapper가 타겟하는 axis.
- copy : bool, default True
  - 데이터도 복사한다.
- inplace : bool, default False
  - 새 데이터프레임을 만드는 대신에, 원본을 수정한다.
- level : int or level name, default None
  - multiIndex의 경우, 해당 레벨의 인덱스 이름만 바꾼다.
- errors : {‘ignore’, ‘raise’}, default ‘ignore’
  - If ‘raise’, raise a KeyError when a dict-like mapper, index, or columns contains labels that are not present in the Index being transformed. If ‘ignore’, existing keys will be renamed and extra keys will be ignored.

In [161]:
df2 = copy.deepcopy(df)
df2 = df2.set_index(['가','나'])
df2

다     라     마          바         사         아
가   나                                                  
1   4    NaN   0.0   NaN   2.000000  2.000000       NaN
100 4    NaN   0.0   NaN  20.000000  2.000000       NaN
1   6    NaN   NaN   NaN   2.449490  2.449490       NaN
    2    3.0   4.0   5.0   1.414214  1.414214   2.44949
    1    1.0   1.0   1.0   1.000000  1.000000   1.00000
    1    1.0   1.0   1.0   1.000000  1.000000   1.00000
22  22  22.0  22.0  22.0  22.000000  4.690416  22.00000

In [165]:
def column_mapper(x):
    if x == '다':
        return 'C'
    elif x == '아':
        return 'H'
    else:
        return x
    
column_mapper2 = {'라':'RARA', '바':'BABA'}

df2_1 = df2.rename(columns=column_mapper)
df2_2 = df2.rename(columns=column_mapper2)
# Display title and DataFrame
display_multiple_dfs([[df2_1, 'Mapper로 함수 사용'], [df2_2, 'Mapper로 dict 사용']], styles)
# mapper에 함수를 할당하면, 모든 이름에 대해 다 적용을 한다.
# 함수에서 대상 이름에 대한 로직이 없다면, None으로 들어가게 된다.

Mapper로 함수 사용 
 
 
 
 
   
   
 C 
 라 
 마 
 바 
 사 
 H 
 
 
 가 
 나 
   
   
   
   
   
   
 
 
 
 
 1 
 4 
 nan 
 0.000000 
 nan 
 2.000000 
 2.000000 
 nan 
 
 
 100 
 4 
 nan 
 0.000000 
 nan 
 20.000000 
 2.000000 
 nan 
 
 
 1 
 6 
 nan 
 nan 
 nan 
 2.449490 
 2.449490 
 nan 
 
 
 2 
 3.000000 
 4.000000 
 5.000000 
 1.414214 
 1.414214 
 2.449490 
 
 
 1 
 1.000000 
 1.000000 
 1.000000 
 1.000000 
 1.000000 
 1.000000 
 
 
 1 
 1.000000 
 1.000000 
 1.000000 
 1.000000 
 1.000000 
 1.000000 
 
 
 22 
 22 
 22.000000 
 22.000000 
 22.000000 
 22.000000 
 4.690416 
 22.000000 
 
 
 

 
 
 
 Mapper로 dict 사용 
 
 
 
 
   
   
 다 
 RARA 
 마 
 BABA 
 사 
 아 
 
 
 가 
 나 
   
   
   
   
   
   
 
 
 
 
 1 
 4 
 nan 
 0.000000 
 nan 
 2.000000 
 2.000000 
 nan 
 
 
 100 
 4 
 nan 
 0.000000 
 nan 
 20.000000 
 2.000000 
 nan 
 
 
 1 
 6 
 nan 
 nan 
 nan 
 2.449490 
 2.449490 
 nan 
 
 
 2 
 3.000000 
 4.000000 
 5.000000 
 1.414214 
 1.414214 
 2.449490 
 
 
 1 
 1.000000 
 1.000000 
 1.000000 
 1.000000 
 1.000000 
 1.000000 
 
 
 1 
 1.000000 
 1.000000 
 1.000000 
 1.000000 
 1.000000 
 1.000000 
 
 
 22 
 22 
 22.000000 
 22.000000 
 22.000000 
 22.000000 
 4.690416 
 22.000000

In [166]:
index_mapper = {1:'IIII', 22:'TTTT'}

In [167]:
df2_1 = df2.rename(index=index_mapper, level='가')
df2_2 = df2.rename(index=index_mapper, level=0)
display_multiple_dfs([[df2_1, 'level 컬럼 레벨 이름으로 세팅'], [df2_2, 'level 컬럼 레벨 숫자로 세팅']], styles)

level 컬럼 레벨 이름으로 세팅 
 
 
 
 
   
   
 다 
 라 
 마 
 바 
 사 
 아 
 
 
 가 
 나 
   
   
   
   
   
   
 
 
 
 
 IIII 
 4 
 nan 
 0.000000 
 nan 
 2.000000 
 2.000000 
 nan 
 
 
 100 
 4 
 nan 
 0.000000 
 nan 
 20.000000 
 2.000000 
 nan 
 
 
 IIII 
 6 
 nan 
 nan 
 nan 
 2.449490 
 2.449490 
 nan 
 
 
 2 
 3.000000 
 4.000000 
 5.000000 
 1.414214 
 1.414214 
 2.449490 
 
 
 1 
 1.000000 
 1.000000 
 1.000000 
 1.000000 
 1.000000 
 1.000000 
 
 
 1 
 1.000000 
 1.000000 
 1.000000 
 1.000000 
 1.000000 
 1.000000 
 
 
 TTTT 
 22 
 22.000000 
 22.000000 
 22.000000 
 22.000000 
 4.690416 
 22.000000 
 
 
 

 
 
 
 level 컬럼 레벨 숫자로 세팅 
 
 
 
 
   
   
 다 
 라 
 마 
 바 
 사 
 아 
 
 
 가 
 나 
   
   
   
   
   
   
 
 
 
 
 IIII 
 4 
 nan 
 0.000000 
 nan 
 2.000000 
 2.000000 
 nan 
 
 
 100 
 4 
 nan 
 0.000000 
 nan 
 20.000000 
 2.000000 
 nan 
 
 
 IIII 
 6 
 nan 
 nan 
 nan 
 2.449490 
 2.449490 
 nan 
 
 
 2 
 3.000000 
 4.000000 
 5.000000 
 1.414214 
 1.414214 
 2.449490 
 
 
 1 
 1.000000 
 1.000000 
 1.000000 
 1.000000 
 1.000000 
 1.000000 
 
 
 1 
 1.000000 
 1.000000 
 1.000000 
 1.000000 
 1.000000 
 1.000000 
 
 
 TTTT 
 22 
 22.000000 
 22.000000 
 22.000000 
 22.000000 
 4.690416 
 22.000000

# MultiIndex
  - MultiIndex는 기본적으로 tuple의 형태를 하고 있다.
  - MultiIndex 사용 시, index를 정렬하지 않으면, `PerformanceWarning: indexing past lexsort depth may impact performance.` 경고가 발생한다.
  - sort_index로 index를 정렬해 주어야 한다.

In [116]:
df2 = df2.sort_index()
df2

다     라     마          바         사         아
가   나                                                  
1   1    1.0   1.0   1.0   1.000000  1.000000   1.00000
    1    1.0   1.0   1.0   1.000000  1.000000   1.00000
    2    3.0   4.0   5.0   1.414214  1.414214   2.44949
    4    NaN   0.0   NaN   2.000000  2.000000       NaN
    6    NaN   NaN   NaN   2.449490  2.449490       NaN
22  22  22.0  22.0  22.0  22.000000  4.690416  22.00000
100 4    NaN   0.0   NaN  20.000000  2.000000       NaN

In [117]:
df2.loc[(1,4),:]

,,다,라,마,바,사,아
가,나,,,,,,
1,4,NaN,0.0,NaN,2.0,2.0,NaN


## 한 레벨의 index만 가져와서 활용하기

In [118]:
print(df2.index.get_level_values(0))
print(df2.index.get_level_values(1))

Int64Index([1, 1, 1, 1, 1, 22, 100], dtype='int64', name='가')
Int64Index([1, 1, 2, 4, 6, 22, 4], dtype='int64', name='나')


In [119]:
# 첫 번째 레벨의 index가 1인 행을 가져옴
df2.loc[df2.index.get_level_values(0) == 1]

다    라    마         바         사        아
가 나                                            
1 1  1.0  1.0  1.0  1.000000  1.000000  1.00000
  1  1.0  1.0  1.0  1.000000  1.000000  1.00000
  2  3.0  4.0  5.0  1.414214  1.414214  2.44949
  4  NaN  0.0  NaN  2.000000  2.000000      NaN
  6  NaN  NaN  NaN  2.449490  2.449490      NaN